In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
from sklearn.ensemble import IsolationForest

In [29]:
data = pd.read_csv("penguins_lter.csv")
colnames = list(data.select_dtypes(["number"]).columns)

In [30]:
species = data["Species"]
island = data["Island"]
cl = data["Culmen Length (mm)"].to_numpy().reshape(1,-1)
cd = data["Culmen Depth (mm)"].to_numpy().reshape(-1,1)
fl = data["Flipper Length (mm)"].to_numpy().reshape(-1,1)
bm = data["Body Mass (g)"].to_numpy().reshape(-1,1)
dc = data["Delta 13 C (o/oo)"].to_numpy().reshape(-1,1)
dn = data["Delta 15 N (o/oo)"].to_numpy().reshape(-1,1)

### Odlehlé hodnoty
Úkol: Zjistěta, zda zvolená datová sada obsahuje nějaké odlehlé hodnoty

In [31]:
data.describe()

,Sample Number,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Delta 15 N (o/oo),Delta 13 C (o/oo)
count,344.000000,342.000000,342.000000,342.000000,342.000000,330.000000,331.000000
mean,63.151163,43.921930,17.151170,200.915205,4201.754386,8.733382,-25.686292
std,40.430199,5.459584,1.974793,14.061714,801.954536,0.551770,0.793961
min,1.000000,32.100000,13.100000,172.000000,2700.000000,7.632200,-27.018540
25%,29.000000,39.225000,15.600000,190.000000,3550.000000,8.299890,-26.320305
50%,58.000000,44.450000,17.300000,197.000000,4050.000000,8.652405,-25.833520
75%,95.250000,48.500000,18.700000,213.000000,4750.000000,9.172123,-25.062050
max,152.000000,59.600000,21.500000,231.000000,6300.000000,10.025440,-23.787670


In [32]:
data.head()

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,11/11/07,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,11/11/07,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,11/16/07,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,NaN
3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,11/16/07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adult not sampled.
4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,11/16/07,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,NaN


In [43]:
def remove_nan(cl):
    return cl[~np.isnan(cl)].reshape(-1,1)

In [54]:
def find_outliers(cl):
    # finding the 1st quartile
    q1 = np.quantile(cl, 0.25)
    
    # finding the 3rd quartile
    q3 = np.quantile(cl, 0.75)
    med = np.median(cl)
    
    # finding the iqr region
    iqr = q3-q1
    
    # finding upper and lower whiskers
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)
    # print(iqr, upper_bound, lower_bound)

    outliers = cl[(cl <= lower_bound) | (cl >= upper_bound)]
    print('The following are the outliers in the boxplot:{}'.format(outliers))

cl = remove_nan(cl)
cd = remove_nan(cd)
fl = remove_nan(fl)
bm = remove_nan(bm)
dc = remove_nan(dc)
dn = remove_nan(dn)

find_outliers(cl)
find_outliers(cd)
find_outliers(fl)
find_outliers(bm)
find_outliers(dc)
find_outliers(dn)

The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]


Datová sada neobsahuje žádné odlehlé hodnoty, pokud zkoumáme všechny druhy tučňáků zároveň. Dále budeme zkoumat odlehlé hodnoty pro každý druh tučňáka zvlášť.

In [55]:
def outliers_for_species(species, data):
    data = data[data["Species"] == species]

    species = data["Species"]
    island = data["Island"]
    cl = data["Culmen Length (mm)"].to_numpy().reshape(1,-1)
    cd = data["Culmen Depth (mm)"].to_numpy().reshape(-1,1)
    fl = data["Flipper Length (mm)"].to_numpy().reshape(-1,1)
    bm = data["Body Mass (g)"].to_numpy().reshape(-1,1)
    dc = data["Delta 13 C (o/oo)"].to_numpy().reshape(-1,1)
    dn = data["Delta 15 N (o/oo)"].to_numpy().reshape(-1,1)

    find_outliers(remove_nan(cl))
    find_outliers(remove_nan(cd))
    find_outliers(remove_nan(fl))
    find_outliers(remove_nan(bm))
    find_outliers(remove_nan(dc))
    find_outliers(remove_nan(dn))

In [56]:
outliers_for_species("Adelie Penguin (Pygoscelis adeliae)", data)

The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[21.5]
The following are the outliers in the boxplot:[172. 210.]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]


In [57]:
outliers_for_species("Chinstrap penguin (Pygoscelis antarctica)", data)

The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[4800. 2700.]
The following are the outliers in the boxplot:[-23.89017 -25.1455  -23.78767]
The following are the outliers in the boxplot:[]


In [58]:
outliers_for_species("Gentoo penguin (Pygoscelis papua)", data)

The following are the outliers in the boxplot:[59.6]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]
The following are the outliers in the boxplot:[]


<b>Adelie Penguin:</b>
* 1 odlehlá hodnota pro culmen depth (21.5)
* 2 odlehlé hodnoty pro flipper length (172 a 210)

<b>Chinstrap Penguin:</b>
* 2 odlehlé hodnoty pro body mass (4800 a 2700)
* 3 odlehlé hodnoty pro delta 13

<b>Gentoo Penguin:</b>
* 1 odlehlé hodnotao pro culmen length (59.6)

### Chybějící hodnoty
Úkol: Proveďte podrobnou analýzu chybějících hodnot (celkový počet chybějících hodnot, počet objektů s více chybějícími hodnotami, atd.)

In [64]:
cl = data["Culmen Length (mm)"].to_numpy().reshape(-1,1)
cd = data["Culmen Depth (mm)"].to_numpy().reshape(-1,1)
fl = data["Flipper Length (mm)"].to_numpy().reshape(-1,1)
bm = data["Body Mass (g)"].to_numpy().reshape(-1,1)
dc = data["Delta 13 C (o/oo)"].to_numpy().reshape(-1,1)
dn = data["Delta 15 N (o/oo)"].to_numpy().reshape(-1,1)

In [76]:
def get_number_of_missing_values(array, name):
    total = array.shape[0]
    missing = array[np.isnan(array)].reshape(-1,1).shape[0]
    print(name, ": ", missing, "/", total)

In [77]:
get_number_of_missing_values(cl, "Culmen length")
get_number_of_missing_values(cd, "Culmen depth")
get_number_of_missing_values(fl, "Flipper length")
get_number_of_missing_values(bm, "Body mass")
get_number_of_missing_values(dc, "Delta 13")
get_number_of_missing_values(dn, "Delta 15")

Culmen length :  2 / 344
Culmen depth :  2 / 344
Flipper length :  2 / 344
Body mass :  2 / 344
Delta 13 :  13 / 344
Delta 15 :  14 / 344


In [87]:
name = data["studyName"]
number = data["Sample Number"]
species = data["Species"]
island = data["Island"]
stage = data["Stage"]
id = data["Individual ID"]
clutch = data["Clutch Completion"]
date = data["Date Egg"]
sex = data["Sex"]
comments = data["Comments"]

In [91]:
def get_number_of_missing_strings(series, name):
    total = len(series)
    missing = series.isnull().sum()
    print(name, ": ", missing, "/", total)

In [93]:
get_number_of_missing_strings(name, "Name")
get_number_of_missing_strings(number, "Sample number")
get_number_of_missing_strings(species, "Species")
get_number_of_missing_strings(island, "Island")
get_number_of_missing_strings(stage, "Stage")
get_number_of_missing_strings(id, "Idividual ID")
get_number_of_missing_strings(clutch, "Clutch completion")
get_number_of_missing_strings(date, "Date egg")
get_number_of_missing_strings(sex, "Sex")
get_number_of_missing_strings(comments, "Comments")

Name :  0 / 344
Sample number :  0 / 344
Species :  0 / 344
Island :  0 / 344
Stage :  0 / 344
Idividual ID :  0 / 344
Clutch completion :  0 / 344
Date egg :  0 / 344
Sex :  10 / 344
Comments :  318 / 344


Celkově je 35 numerických odlehlých hodnot.

Dále chybí 10 hodnot u pohlaví tučňáka a 318 komentářů (což není důležité)

In [97]:
print(data.shape)
not_empty = data.dropna()
print(not_empty.shape())

(13, 17)
(13, 17)
